# CS-5600/6600 Lecture 11 - Dimension Reduction

**Instructor: Dylan Zwick**

*Weber State University*

Reference: [Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/) by Aurélien Géron - [Dimensionality Reduction](https://github.com/ageron/handson-ml3/blob/main/08_dimensionality_reduction.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial.transform import Rotation

from sklearn.decomposition import PCA

from sklearn.datasets import fetch_openml

from sklearn.random_projection import johnson_lindenstrauss_min_dim

from sklearn.random_projection import GaussianRandomProjection

<center>
  <img src="https://drive.google.com/uc?export=view&id=1Ev6HunJV6MsWFmoup3MKlqSFsaFd8DQL" alt="The Curse of Dimensionality">
</center>

I asked ChatGPT to create a Vincent Price horror movie abouth "The Curse of Dimensionality". Not bat, except for, you know the words.

##The Curse of Dimensionality

In Lecture 9, we discussed how we can transform categorical data into numeric data through "one-hot encoding". Basically, we just create a new feature - a new dimension - for each of the possible categories, and give the observation a 1 for whatever category it has, and a 0 for all the others. Suppose we tried to do this for something like words within a document. We could have a column for every word in a dictionary, and a document's values could be the number of times a given word appears within the document. This is something that is actually done - but would have *tens of thousands* of columns. This would mean each document would be some point in a space with tens of thousands of dimensions.

So... what's wrong with that? Well, when you get to very high dimensions, you run into problems. This is knows as the **curse of dimensionality**. (Cue the spooky Halloween music.) What are some of these problems?

Well, suppose you pick a random point in a unit square. It will have only about a .4% chance of being located less than .0001 from a border. So, it's unlikely to be "extreme" along any dimension. In a 10,000-dimensional unit hypercube, this probability is almost 100%. A more significant difference is that if you pick two points randomly in a unit square, the distance between them will be, on average, $0.52$. If you pick two random points in a three-dimensional unit cube, the average distance will be roughly $0.66%$. However, two points picked randomly in a 1,000,000-dimensional unit hypercube, will have an average distance between them of about 408.25. There's just a lot of space in high dimensions, which means that high dimensional datasets are very sparse. This means a new instance will likely be far away from any training instance, which makes predictions more difficult. We make predictions based upon similarities with things we've already seen, and in high dimensions things just aren't that similar.

So, how do we solve this? Well, one way is through that great solution that cures all problems in machine learning - MORE DATA! However, that's not always possible. And, in fact, in high dimensions it's particularly difficult. In 100 dimensions to ensure the training instances are within $0.1$ of each other on average, assuming they were spread out uniformly in all dimensions, you'd need more training instances than atoms in the observable universe.

So, what do we do in practice? We usually find a projection into fewer dimensions.

##Projections

In most real-world problems, training instances are *not* spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated. As a result, all training instances actually lie within (or close to) a much lower-dimensional subspace of the high-dimensional space.

For example, the observations (circles) in the 3D plot below are all pretty close to a plane. So, we can project them down onto this plane (losing a bit of data), and construct a 2D representation that captures most of what we had in 3D.

Let's generate a small 3D dataset. It's an oval shape, rotated in 3D space, with points distributed unevenly, and with quite a lot of noise:

In [ ]:
m = 60
X = np.zeros((m, 3))  # initialize 3D dataset
np.random.seed(42)
angles = (np.random.rand(m) ** 3 + 0.5) * 2 * np.pi  # uneven distribution
X[:, 0], X[:, 1] = np.cos(angles), np.sin(angles) * 0.5  # oval
X += 0.28 * np.random.randn(m, 3)  # add more noise
X = Rotation.from_rotvec([np.pi / 29, -np.pi / 20, np.pi / 4]).apply(X)
X += [0.2, 0, 0.2]  # shift a bit

In [ ]:
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)  # dataset reduced to 2D
X3D_inv = pca.inverse_transform(X2D)  # 3D position of the projected samples
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)

axes = [-1.4, 1.4, -1.4, 1.4, -1.1, 1.1]
x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 10),
                     np.linspace(axes[2], axes[3], 10))
w1, w2 = np.linalg.solve(Vt[:2, :2], Vt[:2, 2])  # projection plane coefs
z = w1 * (x1 - pca.mean_[0]) + w2 * (x2 - pca.mean_[1]) - pca.mean_[2]  # plane
X3D_above = X[X[:, 2] >= X3D_inv[:, 2]]  # samples above plane
X3D_below = X[X[:, 2] < X3D_inv[:, 2]]  # samples below plane

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection="3d")

# plot samples and projection lines below plane first
ax.plot(X3D_below[:, 0], X3D_below[:, 1], X3D_below[:, 2], "ro", alpha=0.3)
for i in range(m):
    if X[i, 2] < X3D_inv[i, 2]:
        ax.plot([X[i][0], X3D_inv[i][0]],
                [X[i][1], X3D_inv[i][1]],
                [X[i][2], X3D_inv[i][2]], ":", color="#F88")

ax.plot_surface(x1, x2, z, alpha=0.1, color="b")  # projection plane
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "b+")  # projected samples
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "b.")

# now plot projection lines and samples above plane
for i in range(m):
    if X[i, 2] >= X3D_inv[i, 2]:
        ax.plot([X[i][0], X3D_inv[i][0]],
                [X[i][1], X3D_inv[i][1]],
                [X[i][2], X3D_inv[i][2]], "r--")

ax.plot(X3D_above[:, 0], X3D_above[:, 1], X3D_above[:, 2], "ro")

def set_xyz_axes(ax, axes):
    ax.xaxis.set_rotate_label(False)
    ax.yaxis.set_rotate_label(False)
    ax.zaxis.set_rotate_label(False)
    ax.set_xlabel("$x_1$", labelpad=8, rotation=0)
    ax.set_ylabel("$x_2$", labelpad=8, rotation=0)
    ax.set_zlabel("$x_3$", labelpad=8, rotation=0)
    ax.set_xlim(axes[0:2])
    ax.set_ylim(axes[2:4])
    ax.set_zlim(axes[4:6])

set_xyz_axes(ax, axes)
ax.set_zticks([-1, -0.5, 0, 0.5, 1])

plt.show()

Here's the projection viewed as a 2D image:

In [ ]:
# extra code – this cell generates and saves Figure 8–3

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect='equal')
ax.plot(X2D[:, 0], X2D[:, 1], "b+")
ax.plot(X2D[:, 0], X2D[:, 1], "b.")
ax.plot([0], [0], "bo")
ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True,
         head_length=0.1, fc='b', ec='b', linewidth=4)
ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True,
         head_length=0.1, fc='b', ec='b', linewidth=1)
ax.set_xlabel("$z_1$")
ax.set_yticks([-0.5, 0, 0.5, 1])
ax.set_ylabel("$z_2$", rotation=0)
ax.set_axisbelow(True)
ax.grid(True)
plt.show()

How did we get this plane? Using something called *principal component analysis* (PCA). The idea behind principle component analysis is that you find the linear subspace (or *hyperplane*) that maximizes the *variance* (the spread) of the projection.

###Principal Component Analysis

This is perhaps easiest to understand in the context of a projection onto a line. Suppose you've got a set of observations in two dimensions:

In [ ]:
angle = np.pi / 5
stretch = 5
m = 200

np.random.seed(3)
X_line = np.random.randn(m, 2) / 10
X_line = X_line @ np.array([[stretch, 0], [0, 1]])  # stretch
X_line = X_line @ [[np.cos(angle), np.sin(angle)],
                   [np.sin(angle), np.cos(angle)]]  # rotate

u1 = np.array([np.cos(angle), np.sin(angle)])
u2 = np.array([np.cos(angle - 2 * np.pi / 6), np.sin(angle - 2 * np.pi / 6)])
u3 = np.array([np.cos(angle - np.pi / 2), np.sin(angle - np.pi / 2)])

X_proj1 = X_line @ u1.reshape(-1, 1)
X_proj2 = X_line @ u2.reshape(-1, 1)
X_proj3 = X_line @ u3.reshape(-1, 1)

plt.plot(X_line[:, 0], X_line[:, 1], "ro", alpha=0.5)
plt.show()

There are many lines upon which we could project these points, and these projections would give us many distributions:

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot2grid((3, 2), (0, 0), rowspan=3)
plt.plot([-1.4, 1.4], [-1.4 * u1[1] / u1[0], 1.4 * u1[1] / u1[0]], "k-",
         linewidth=2)
plt.plot([-1.4, 1.4], [-1.4 * u2[1] / u2[0], 1.4 * u2[1] / u2[0]], "k--",
         linewidth=2)
plt.plot([-1.4, 1.4], [-1.4 * u3[1] / u3[0], 1.4 * u3[1] / u3[0]], "k:",
         linewidth=2)
plt.plot(X_line[:, 0], X_line[:, 1], "ro", alpha=0.5)
plt.arrow(0, 0, u1[0], u1[1], head_width=0.1, linewidth=4, alpha=0.9,
          length_includes_head=True, head_length=0.1, fc="b", ec="b", zorder=10)
plt.arrow(0, 0, u3[0], u3[1], head_width=0.1, linewidth=1, alpha=0.9,
          length_includes_head=True, head_length=0.1, fc="b", ec="b", zorder=10)
plt.text(u1[0] + 0.1, u1[1] - 0.05, r"$\mathbf{c_1}$", color="blue")
plt.text(u3[0] + 0.1, u3[1], r"$\mathbf{c_2}$", color="blue")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$", rotation=0)
plt.axis([-1.4, 1.4, -1.4, 1.4])
plt.grid()

plt.subplot2grid((3, 2), (0, 1))
plt.plot([-2, 2], [0, 0], "k-", linewidth=2)
plt.plot(X_proj1[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid()

plt.subplot2grid((3, 2), (1, 1))
plt.plot([-2, 2], [0, 0], "k--", linewidth=2)
plt.plot(X_proj2[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid()

plt.subplot2grid((3, 2), (2, 1))
plt.plot([-2, 2], [0, 0], "k:", linewidth=2)
plt.plot(X_proj3[:, 0], np.zeros(m), "ro", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.axis([-2, 2, -1, 1])
plt.xlabel("$z_1$")
plt.grid()

plt.show()

The idea behind principal component analysis is we want to find the projection that preserves the greatest distinction among our observations. More precisely, we want to find the projection that maximizes the *variance* (the spread around the mean). Taking a look at our dots above, we see that projection along the solid line keeps the points spread out much more than projection along the dashed line. So, the solid line would be preferable, and is in fact the line we would get through principle component analysis.

Please note there is a lot of very cool linear algebra involved in principal component analysis. Specifically, it's an example of something called a *singular value decomposition*. We won't be able to go over this in detail in this class, but we definitely will in the upcoming math for data science class.

More geneally, the first principle component is the line that, if you project onto it, preserves the most variance. The second principle component is the line *perpendicular to the first* that preserves the most variance, which would be the dotted line above. In two dimensions there will only be two principle components, but in three dimensions there will be three, and you can form a projection plane from the first two.

Let's see how we'd use PCA for our 3D dataset created above.

In [ ]:
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

We can check out the principle components using the *components* attribute of our pca object

In [ ]:
pca.components_

A useful bit of information about the principle components is the *explained variance ratio*, which tells you the percentage of the variance of your original dataset that is explained by the various principal components.

In [ ]:
pca.explained_variance_ratio_

So, the two principal components explain:

In [ ]:
pca.explained_variance_ratio_.sum()

A bit more than 90% of all the variance.

####Choosing the right number of components

Instead of arbitrarily choosing the number of dimensions to reduce down to, it might be better to choose the number of dimensinos that add up to a sufficiently large portion of the variance - say $95\%$. An exception here is if you're reducing dimensionality for data visualization, in which case you'd want to limit to 2 or 3. Probably 2.

For example, we can do principal component analysis on the MNIST dataset to see how many components we need to explain 95% of the variance.

In [ ]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")
X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) +1
d

This is significantly less than 784, the number of dimensions in our dataset. However, an easier way of doing this would be to set the 95% value when you create the PCA object:

In [ ]:
pca = PCA(n_components = 0.95)
X_reduced = pca.fit_transform(X_train)

The actual number of components here will be determined during training, and is stored in the *n_components_* attribute:

In [ ]:
pca.n_components_

You can also plot the explained variance of the number of dimensions, and pick out where it tapers off:

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.plot(d, 0.95, "ko")
plt.annotate("Elbow", xy=(65, 0.85), xytext=(70, 0.7),
             arrowprops=dict(arrowstyle="->"))
plt.grid(True)
plt.show()

###Reconstructing The Data From The Principlal Components

We can decompress the principle components back to the original number of dimensions by applying the inverse transformation of the PCA projection. This won't give you back the original data (you lose some when you project), but you can get close. For example, doing this for our MNIST data we get:

In [ ]:
X_recovered = pca.inverse_transform(X_reduced)

plt.figure(figsize=(7, 4))
for idx, X in enumerate((X_train[::2100], X_recovered[::2100])):
    plt.subplot(1, 2, idx + 1)
    plt.title(["Original", "Compressed"][idx])
    for row in range(5):
        for col in range(5):
            plt.imshow(X[row * 5 + col].reshape(28, 28), cmap="binary",
                       vmin=0, vmax=255, extent=(row, row + 1, col, col + 1))
            plt.axis([0, 5, 0, 5])
            plt.axis("off")

Pretty close!

###Random Projection

There's another type of projection that has no right to work as well as it  does. Namely, *random* projection. It's a bit weird, but random projections are likely to preserve distances fairly well. So, two similar instances will likely remain similar after a random projection, while two different instances will likely remain different.

The more dimensions you drop, the more information you lose. Johnson and Lindenstrauss came up with an equation that determines the minumum number of dimensions to preserve in order to ensure, with high probability, that distances won't change by more than a given tolerance. This number is calculated with a function in sklearn.

In [ ]:
m, e = 5000, .1 #Here m is the number of observations, and e is the tolerance. Note this is independent of the original number of dimensions.
d = johnson_lindenstrauss_min_dim(m, eps=e)
d

We can just generate a random matrix $P$ of shape $[d,n]$, where each item is sampled randomly from a Gaussian distribution with mean $0$ and variance $1/d$, and use it to project a dataset from $n$ dimensions down to $d$:

In [ ]:
n = 20000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d)
X = np.random.randn(m,n) #Generate a fake dataset
X_reduced = X @ P.T #Project it down to d dimensions

That's it! Simple, efficient, and no training is required.

Scikit-Learn offers a *GaussianRandomProjection* class to do exactly what we just did with its fit method.

In [ ]:
gaussian_rnd_proj = GaussianRandomProjection(eps=e, random_state=42)
X_reduced_2 = gaussian_rnd_proj.fit_transform(X)

Scikit-Learn also provides a second random projection transformer, knows as *SparseRandomProjection*. It determines the target dimensionality in the same way, genarets a random matrix of the same shape, and performs the projection identically. The main difference is the random matrix is sparse. It's usually preferable to use this method, as its much cheaper and uses much less space. The ratio $r$ of nonzero items is the spare random matrix is called its *density*. By default, it's equal to $1/\sqrt{n}$.

##References

* [PCA from Statquest](https://www.youtube.com/watch?v=FgakZw6K1QQ)
* [Original PCA paper from 1901](https://zenodo.org/records/1430636)